In [ ]:
#@title Check the GPU
!nvidia-smi

In [ ]:
#@title Installs mamba and updates base

%%shell
curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh
chmod +x Mambaforge-Linux-x86_64.sh
bash Mambaforge-Linux-x86_64.sh -b -f -p /usr/local
mamba config --env --set always_yes true
rm Mambaforge-Linux-x86_64.sh
mamba update -n base -c defaults conda -y

In [ ]:
#@title Clone github repository
!git clone https://github.com/justinjohn0306/so-vits-svc -b 4.0-v2
%cd /content/so-vits-svc

In [ ]:
#@title Creates mamba environment and activates it from environment.yaml. Takes ~8 minutes.

%%shell
mamba env create -f environment.yaml
mamba init bash
source /root/.bashrc
source activate sovits
pip install pyworld praat-parselmouth fairseq

In [ ]:
#@title Download necessary model files
# 源仓库地址：[contentvec](https://github.com/auspicious3000/contentvec)
# 模型原下载链接：[checkpoint_best_legacy_500.pt](https://ibm.box.com/s/z1wgl1stco8ffooyatzdwsqn2psd9lrr)
# 由于源网盘无法提供http直链，根据mit协议，对模型进行二次分发，提供下载直链
!wget -P hubert/ https://huggingface.co/innnky/contentvec/resolve/main/checkpoint_best_legacy_500.pt

# Dataset preprocessing

In [ ]:
#@title Loading Google Drive
#@markdown load Google Drive
from google.colab import drive
drive.mount('/content/drive')

This processing script can preprocess multiple speakers at one time, and generate multi-speaker filelists and corresponding configuration files

Just put your dataset into the dataset_raw directory according to the following file structure


```
dataset_raw
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav

```




In [ ]:
#@title Load the packaged dataset from Google Cloud Disk for preprocessing
#@markdown **The data set of sovits3.0 no longer needs a specific file structure. Put all the wav files of the data set in the same folder, compress them into zip and upload them to Google Cloud Disk. This processing script can be used once When processing multiple datasets, please decompress each dataset in turn**

#@markdown dataset name (**character’s English/Pinyin name**, which is the same as when creating the data folder; without zip.)
DATASETNAME = "kiritan"  #@param {type:"string"}
#@markdown Compressed package path (Google disk path, don’t change this if you pass it to dataset, create a new one if there is no dataset folder)
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_NAME = ZIP_PATH + DATASETNAME

!unzip -d /content/so-vits-svc/dataset_raw {ZIP_NAME}.zip

In [ ]:
#@title Resample audio to 44100hz
!python resample.py

In [ ]:
#@title Divide training set and Generate configuration file
!python preprocess_flist_config.py

In [ ]:
#@title Generate hubert and f0
!python preprocess_hubert_f0.py

In [ ]:
#@title At this point, the preprocessing of the data set is completed, and the data set and related files are saved to the dataset folder of Google Cloud Disk, which is convenient for the next training
#Compress the dataset folder
!zip -r dataset.zip /content/so-vits-svc/dataset
#@markdown Customize the name of the dataset folder under the dataset folder of Google Cloud Disk to avoid confusion
dataset_name_drive = "kiritan_preprocessed"  #@param {type:"string"}
DATASET_PATH_DRIVE = "/content/drive/MyDrive/dataset/" + dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp /content/so-vits-svc/dataset.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.txt "{DATASET_PATH_DRIVE}"
!cp filelists/val.txt "{DATASET_PATH_DRIVE}"

In [ ]:
#@title If the data set has been preprocessed, you can skip the preprocessing part and decompress the processed data and configuration files directly from the cloud disk
#@markdown Load the preprocessed data set from Google Cloud Disk, the folder name is the same as what you entered when you backed up
back_up_name = "kiritan_preprocessed"  #@param {type:"string"}
BACK_UP_DATASET_PATH = "/content/drive/MyDrive/dataset/" + back_up_name
!unzip {BACK_UP_DATASET_PATH}/dataset.zip -d /
!cp {BACK_UP_DATASET_PATH}/config.json /content/so-vits-svc/configs/config.json 
!cp {BACK_UP_DATASET_PATH}/val.txt filelists/val.txt
!cp {BACK_UP_DATASET_PATH}/train.txt filelists/train.txt


# 拷贝云盘上保存的记录点
# !cp /content/drive/MyDrive/G_800.pth logs/48k/
# !cp /content/drive/MyDrive/D_800.pth logs/48k/

# Train

In [ ]:
#@title Choose whether to save the trained model to Google Cloud Disk and whether to use the pre-model

#@markdown **Save the trained model file to Google Cloud Disk. After checking, you need to check and execute it when resuming training**
Save_to_drive = True #@param {type:"boolean"}
if Save_to_drive:
  !rm -rf /content/so-vits-svc/logs/44k
  !mkdir -p /content/drive/MyDrive/44k
  !ln -s /content/drive/MyDrive/44k /content/so-vits-svc/logs/44k

#@markdown **Download the pre-model for the first training and continue training after using the record points saved by yourself, no need to download again**

#@markdown **Use the pre-model, check the box below to automatically download and enable**
pre_pth = True #@param {type:"boolean"}
if pre_pth:
  !wget -P logs/44k/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4.0-v2/G_0.pth
  !wget -P logs/44k/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4.0-v2/D_0.pth


In [ ]:
#@title Start Training

#@markdown **start training**

#@markdown **Enable tensorboard to visualize data**
tensorboard_on = True #@param {type:"boolean"}
if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

!python train.py -c configs/config.json -m 44k


In [ ]:
#@title Manually back up the trained model files to Google Cloud Disk
#@markdown You need to check the file name of the model under the /content/so-vits-svc/logs/44k/ folder by yourself, and manually modify the file name at the end of the command below
!mv /content/so-vits-svc/logs/44k/G_1000.pth /content/drive/MyDrive
!mv /content/so-vits-svc/logs/44k/D_1000.pth /content/drive/MyDrive

# Reasoning

In [ ]:
#@title Synthetic Audio (Inference)
#@markdown Upload the audio to the ``so-vits-svc/raw`` folder, and then set the model path, configuration file path, and synthesized audio name

!python inference_main.py -m "logs/44k/G_1600.pth" -c "configs/config.json" -n "君の知らない物語-src.wav" -s kiritan
